In [ ]:
import os
os.environ['nnUNet_raw'] = "/content/drive/MyDrive/TFM/raw"
os.environ['nnUNet_preprocessed'] =  "/content/drive/MyDrive/TFM/preprocessed"
os.environ['nnUNet_results'] = "/content/drive/MyDrive/TFM/results"

#Data Processing

During my TFM, I had to modify the Dataset several times for logistical reasons or due to a bad performance of the U-Net. Here we describe the different pieces of code used for that purpose.

## Merging the three masks in one file
Originally, the masks where separated in three masks:
1. A mask with all the lesions: SELs, non-SELs and possible SELs. All of them with a mask value of 1.
2. A mask with only the non-SELs.
3. A mask with only the SELs.

This code creates a nifti with the three of the masks.

In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import glob
import re
import numpy as np

ssd_path = os.path.join(os.sep, "Volumes", "SSD-TOSHIBA")

all_lesions = sorted(glob.glob(os.path.join(ssd_path, "Dataset TFM", "labelsTr", "rMSVIS_*_T1_lesion_mask.nii.gz")))
masks_SEL = sorted(glob.glob(os.path.join(ssd_path, "Dataset TFM", "labelsTr", "rMSVIS_*_T1_SELs_mask.nii.gz")))
masks_nonSEL = sorted(glob.glob(os.path.join(ssd_path, "Dataset TFM", "labelsTr", "rMSVIS_*_T1_nonSELs_mask.nii.gz")))

In [ ]:
def load_nifti(path):
    return nib.load(path).get_fdata()

def extract_value_from_string(file_name):
    # Define the pattern using regular expression
    pattern = r"rMSVIS_(\d+)_T1_nonSELs_mask.nii.gz"

    # Use regular expression to find the value
    match = re.search(pattern, file_name)

    if match:
        extracted_value = match.group(1)
        return extracted_value
    else:
        return None

for sel, no_sel, full in zip(masks_SEL, masks_nonSEL, all_lesions):
    variable = extract_value_from_string(os.path.basename(no_sel))

    # If we already have the mask called "all_labels_mask" we skip this iteration
    if os.path.exists(os.path.join(ssd_path,  "Dataset TFM", "labelsTr", f"rMSVIS_{variable}_T1_all_labels_mask.nii.gz")):
        continue

    print(f"Working with {variable}")

    sel_mask = load_nifti(sel)
    no_sel_mask = load_nifti(no_sel)
    full_mask = load_nifti(full)

    # Convert in zero the values in full mask present in sel mask
    full_mask[sel_mask != 0] = 0
    # Convert in zero the values in full mask present in no sel mask
    full_mask[no_sel_mask != 0] = 0

    # We create a mask with all labels at the same time
    all_labels = sel_mask + no_sel_mask + full_mask

    # Store the new mask
    new_mask = nib.Nifti1Image(full_mask, affine=None)
    all_labels_mask = nib.Nifti1Image(all_labels, affine=None)
    nib.save(new_mask, os.path.join(ssd_path,  "Dataset TFM", "labelsTr", f"rMSVIS_{variable}_T1_possibleSELs_mask.nii.gz"))
    nib.save(all_labels_mask, os.path.join(ssd_path,  "Dataset TFM", "labelsTr", f"rMSVIS_{variable}_T1_all_labels_mask.nii.gz"))

    print(f"The unique values in the mask are: {np.unique(all_labels)}")
    print(f"Done with {variable}")

In [ ]:
all_labels = sorted(glob.glob(os.path.join(ssd_path, "Dataset TFM", "labelsTr", "rMSVIS_*_T1_all_labels_mask.nii.gz")))
imgs = sorted(glob.glob(os.path.join(ssd_path, "Dataset TFM", "imagesTr", "rMSVIS_*_*_T1.nii.gz")))


In [ ]:
# We store the images in the folder DatasetPrepared/Images
from collections import defaultdict

file_groups = defaultdict(list)

# Extract and group files based on the alphanumeric (\w) part
for file in imgs:
    match = re.search(r'rMSVIS_(\w+)_([0-9]{2})_T1.nii.gz', file)
    if match:
        id, timepoint = int(match.group(1)), int(match.group(2))
        file_groups[id].append((timepoint, file))

# Sort and rename the files within each group
for id, files in file_groups.items():
    files.sort(key=lambda x: x[0])  # Sort by the timepoint part

    # Renaming logic
    for index, (timepoint, file) in enumerate(files):
        new_id = "{:03d}".format(id)
        new_timepoint = "{:04d}".format(index)
        new_name = f"rMSVIS_{new_id}_{new_timepoint}.nii.gz"
        #os.rename(file, os.path.join("DatasetPrepared", "Images", new_name))
        os.rename(file, os.path.join(ssd_path, "Dataset TFM", "imagesTr", new_name))

In [ ]:
# We store the masks in the folder DatasetPrepared/Masks
file_groups = defaultdict(list)

for file in all_labels:
    match = re.search(r'rMSVIS_(\w+)_T1_all_labels_mask.nii.gz', file)
    if match:
        id = int(match.group(1))
        file_groups[id].append(file)

# Sort and rename the files within each group
for id, files in file_groups.items():
    files.sort(key=lambda x: x[0])  # Sort by the timepoint part

    # Renaming logic
    for index, file in enumerate(files):
        new_id = "{:03d}".format(id)
        new_name = f"rMSVIS_{new_id}.nii.gz"
        #os.rename(file, os.path.join("DatasetPrepared", "Masks", new_name))
        os.rename(file, os.path.join(ssd_path, "Dataset TFM", "labelsTr", new_name))

## Nifti to NRRD (check in Macbook)


The framework of nnU-Net was giving problems to read the dataset in Nifti format due to the lack of concordance between origins and orientations in the different timepoints or subjects.

In [ ]:
import itk
import glob
import numpy as np
import os

# Images in SSD
ssd_path = "/Volumes/SSD-TOSHIBA/Dataset TFM"
images = sorted(glob.glob(os.path.join(ssd_path, "imagesTr copia", "*.nii.gz")))
masks = sorted(glob.glob(os.path.join(ssd_path, "labelsTr copia", "*.nii.gz")))



In [ ]:
# We store the nifti images into NRRD

for i in range(len(images)):
    img = itk.imread(images[i])
    img_np = itk.array_from_image(img)
    img_itk = itk.GetImageFromArray(img_np)
    itk.imwrite(img_itk, images[i].replace(".nii.gz", ".nrrd"))

In [ ]:
# We store the nifti masks into NRRD
for i in range(len(masks)):
    img = itk.imread(masks[i])
    img_np = itk.array_from_image(img)
    img_itk = itk.GetImageFromArray(img_np)
    itk.imwrite(img_itk, masks[i].replace(".nii.gz", ".nrrd"))

## Changing the possible SEL to non-SEL and substraction between follow-up 2 and screening.
The U-Net presented low values on the validation Dice, as well as in the training. Thus, the possible SELs where changed to non-SEL.

In [ ]:
# Directory to Dataset111
dataset_dir = os.path.join(os.environ["nnUNet_preprocessed"], "Dataset111_SEL",
                           "nnUNetPlans_2d")

segs = glob.glob(os.path.join(dataset_dir, "rMSVIS_*_seg.npy"))
segs.sort()

pattern = re.compile(r'rMSVIS_\d+\.npy')
imgs = [file for file in glob.glob(os.path.join(dataset_dir, "*")) if pattern.match(os.path.basename(file))]
imgs.sort()

dataset_path = os.path.join(os.environ["nnUNet_raw"], "Dataset115_Substract-binary-possibleSEL2noSEL")

if not os.path.exists(os.path.join(dataset_path, "imagesTr")):
  os.makedirs(os.path.join(dataset_path, "imagesTr"))

if not os.path.exists(os.path.join(dataset_path, "labelsTr")):
  os.makedirs(os.path.join(dataset_path, "labelsTr"))


for seg, img in zip(segs, imgs):
  image = np.load(os.path.join(img))
  segmentation = np.load(os.path.join(seg))
  segmentation[segmentation==-1] = 0
  segmentation[segmentation==1] = 1
  segmentation[segmentation==2] = 1
  segmentation[segmentation==3] = 2
  substracted_img = image[2] - image[0]

  sustract_name = os.path.splitext(os.path.basename(img))[0] + "_0001" + ".nii.gz"
  sustract_path = os.path.join(dataset_path, "imagesTr", sustract_name)
  img_name = os.path.splitext(os.path.basename(img))[0] + "_0000" + ".nii.gz"
  img_path = os.path.join(dataset_path, "imagesTr", img_name)

  seg_name = os.path.splitext(os.path.basename(seg))[0] + ".nii.gz"
  seg_path = os.path.join(dataset_path, "labelsTr", seg_name)
  seg_path = seg_path.replace("_seg", "")
  nib.save(nib.Nifti1Image(substracted_img, affine=np.eye(4)), sustract_path)
  nib.save(nib.Nifti1Image(image[0], affine=np.eye(4)), img_path)
  nib.save(nib.Nifti1Image(segmentation[0], affine=np.eye(4)), seg_path)

  print(f"Case {img_name} done")

## Patches (using only SEL and non-SEL)
With the aim of improving the Dice, we create patches around each of the lesions. The patches are all re-formatted to 32x32x32.

In [ ]:
path2imgs = os.path.join(os.environ['nnUNet_raw'], "Dataset115_Substract-binary-possibleSEL2noSEL", "imagesTr")
path2labels = os.path.join(os.environ['nnUNet_raw'], "Dataset115_Substract-binary-possibleSEL2noSEL", "labelsTr")

segs = glob.glob(os.path.join(path2labels, "rMSVIS*.nii.gz"))
img_0 = glob.glob(os.path.join(path2imgs, "rMSVIS_*_0000.nii.gz"))
img_1 = glob.glob(os.path.join(path2imgs, "rMSVIS_*_0001.nii.gz"))

def crop_around_segmentation(image_data, substract_data, segmentation_data, patch_size=(32, 32, 32)):
    # Find connected components in the segmentation mask
    labeled_segments, num_labels = ndimage.label(segmentation_data)

    # Crop patches around each lesion
    patches_img = []
    patches_seg = []
    patches_substract = []
    i = 0
    for label in range(1, num_labels + 1):
        # Extract coordinates of the current lesion
        indices = np.where(labeled_segments == label)

        # Calculate bounding box around the lesion
        min_x, min_y, min_z = np.min(indices, axis=1)
        max_x, max_y, max_z = np.max(indices, axis=1)
        if i == 0:
          print(f"{min_x}, {min_y}, {min_z}")
          print(f"{max_x}, {max_y}, {max_z}")
          i = 1

        # Crop the patch around the lesion
        patch_img = image_data[min_x:max_x + 1, min_y:max_y + 1, min_z:max_z + 1]
        patch_seg = segmentation_data[min_x:max_x + 1, min_y:max_y + 1, min_z:max_z + 1]
        patch_substract = substract_data[min_x:max_x + 1, min_y:max_y + 1, min_z:max_z + 1]
        # Resize the patch to the specified size
        patch_img = ndimage.zoom(patch_img, np.array(patch_size) / patch_img.shape, order=0)
        patch_seg = ndimage.zoom(patch_seg, np.array(patch_size) / patch_seg.shape, order=0)
        patch_substract = ndimage.zoom(patch_substract, np.array(patch_size) / patch_substract.shape, order=0)

        # Append the patch to the list
        patches_img.append(patch_img)
        patches_seg.append(patch_seg)
        patches_substract.append(patch_substract)

    return patches_img, patches_substract, patches_seg

In [ ]:
segs = glob.glob(os.path.join(path2labels, "rMSVIS*.nii.gz"))
img_0 = glob.glob(os.path.join(path2imgs, "rMSVIS_*_0000.nii.gz"))
img_1 = glob.glob(os.path.join(path2imgs, "rMSVIS_*_0001.nii.gz"))

# We declare the paths where we will save the patches
path2patches = os.path.join(os.environ['nnUNet_raw'], "Dataset116_Patches")
path2patches_img = os.path.join(path2patches, "imagesTr")
path2patches_seg = os.path.join(path2patches, "labelsTr")

# We create the folders if they don't exist
if not os.path.exists(path2patches_img):
  os.makedirs(path2patches_img)

if not os.path.exists(path2patches_seg):
    os.makedirs(path2patches_seg)


i = 0
for i0, i1, seg in zip(img_0, img_1, segs):
    if i == 0:
        print(i0)
        print(i1)
        print(seg)
    image_0 = nib.load(i0).get_fdata()
    image_1 = nib.load(i1).get_fdata()
    segmentation = nib.load(seg).get_fdata()

    patches_img_0, patches_img_1, patches_seg = crop_around_segmentation(image_0, image_1, segmentation)
    # We save the patches with a pattern that will go from 000 to 9999.
    # Series will maintain their name: 0000 for image_0, 0001 for image_1

    for patch_img_0, patch_img_1, patch_seg in zip(patches_img_0, patches_img_1, patches_seg):
        patch_name = "patch_" + str(i).zfill(3) + "_0000" ".nii.gz"
        patch_path_img_0 = os.path.join(path2patches_img, patch_name)
        patch_path_img_1 = os.path.join(path2patches_img, patch_name.replace("_0000", "_0001"))
        patch_path_seg = os.path.join(path2patches_seg, patch_name)

        nib.save(nib.Nifti1Image(patch_img_0, affine=np.eye(4)), patch_path_img_0)
        nib.save(nib.Nifti1Image(patch_img_1, affine=np.eye(4)), patch_path_img_1)
        nib.save(nib.Nifti1Image(patch_seg, affine=np.eye(4)), patch_path_seg)

        print(f"Patch {patch_name} done")

        i += 1


In [ ]:
# Since there are too much cases (>5000), we select 200: 100 for non-SEL and 100 for SEL.

# From the cases with label = 1 and the cases with label = 2, we will select 100 cases for each of them.
path2patches_img = os.path.join(path2patches, "imagesTr")
path2patches_seg = os.path.join(path2patches, "labelsTr")

patches_0 = glob.glob(os.path.join(path2patches_img, "patch_*_0000.nii.gz"))
patches_1 = glob.glob(os.path.join(path2patches_img, "patch_*_0001.nii.gz"))
patches_0.sort()
patches_1.sort()

patches_seg = glob.glob(os.path.join(path2patches_seg, "patch_*.nii.gz"))
patches_seg.sort()

path2patches_200 = os.path.join(os.environ['nnUNet_raw'], "Dataset118_Patches_200")
path2patches_img_200 = os.path.join(path2patches_200, "imagesTr")
path2patches_seg_200 = os.path.join(path2patches_200, "labelsTr")


if not os.path.exists(path2patches_img_200):
    os.makedirs(path2patches_img_200)

if not os.path.exists(path2patches_seg_200):
    os.makedirs(path2patches_seg_200)


N=100
# We will select 383 cases with label = 1 and 500 cases with label = 2
i = 0
j = 0
k = 0

for patch_0, patch_1, patch_seg in zip(patches_0, patches_1, patches_seg):
    segmentation = nib.load(patch_seg).get_fdata()
    unique_values = np.unique(segmentation)

    if 1 in unique_values and 2 in unique_values:
        continue
    elif 1 in unique_values and 2 not in unique_values and i < N:
        patch_name = "patch_" + str(k).zfill(3) + "_0000.nii.gz"
        patch_seg_name = "patch_" + str(k).zfill(3) + ".nii.gz"
        patch_path_img = os.path.join(path2patches_img_200, patch_name)
        patch_path_seg = os.path.join(path2patches_seg_200, patch_seg_name)

        #nib.save(nib.load(patch_0), patch_path_img)
        #nib.save(nib.load(patch_1), patch_path_img.replace("_0000", "_0001"))
        nib.save(nib.load(patch_seg), patch_path_seg)

        print(f"Patch {patch_name} done as label 1")

        i += 1
        k += 1
    elif 2 in unique_values and 1 not in unique_values and j < N:
        patch_name = "patch_" + str(k).zfill(3) + "_0000.nii.gz"
        patch_seg_name = "patch_" + str(k).zfill(3) + ".nii.gz"
        patch_path_img = os.path.join(path2patches_img_200, patch_name)
        patch_path_seg = os.path.join(path2patches_seg_200, patch_seg_name)

        #nib.save(nib.load(patch_0), patch_path_img)
        #nib.save(nib.load(patch_1), patch_path_img.replace("_0000", "_0001"))
        nib.save(nib.load(patch_seg), patch_path_seg)

        print(f"Patch {patch_name} done as label 2")

        j += 1
        k += 1
    elif i == N and j == N:
        break
    else:
        continue